Tugas 2 Olah Data Menggunakan Crisp DM

Peran Data Mining Yang Saya Gunakan Adalah Clustering, Assosiaction

In [27]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [28]:
file_path = 'perfume_database.xlsx'
perfume_data = pd.read_excel(file_path)

In [29]:
perfume_data.head()

,brand,perfume,image,launch_year,main_accords,notes,longevity,sillage
0,18 21 Man Made,Sweet Tobacco Spirits,o.37691.jpg,2016.0,"[""wine"", ""vanilla"", ""sweet"", ""woody"", ""aromatic""]","[""Citruses"", ""Saffron"", ""Tonka Bean"", ""Vanilla...","[""4"", ""0"", ""9"", ""14"", ""31""]","[""10"", ""19"", ""28"", ""11""]"
1,40 Notes Perfume,Cashmere Musk,o.15537.jpg,NaN,"[""woody"", ""musky"", ""balsamic"", ""powdery"", ""war...","[""Sandalwood"", ""Cedar"", ""White Musk"", ""Cashmer...","[""1"", ""0"", ""0"", ""1"", ""1""]","[""1"", ""3"", ""0"", ""2""]"
2,40 Notes Perfume,Exotic Ylang Ylang,o.15536.jpg,NaN,"[""yellow floral"", ""white floral"", ""sweet"", ""mu...","[""Ylang-Ylang"", ""Gardenia"", ""Musk""]","[""1"", ""0"", ""0"", ""0"", ""2""]","[""0"", ""2"", ""1"", ""2""]"
3,40 Notes Perfume,Exquisite Amber,o.15538.jpg,NaN,"[""balsamic"", ""vanilla"", ""amber"", ""musky"", ""war...","[""Labdanum"", ""Styrax"", ""Benzoin"", ""Vanilla"", ""...","[""0"", ""0"", ""0"", ""0"", ""1""]","[""1"", ""0"", ""3"", ""1""]"
4,40 Notes Perfume,Oudwood Veil,o.15539.jpg,NaN,"[""oud"", ""amber"", ""fresh spicy"", ""balsamic"", ""w...","[""Kephalis"", ""Agarwood (Oud)""]","[""1"", ""1"", ""1"", ""0"", ""6""]","[""1"", ""4"", ""0"", ""5""]"


In [30]:
perfume_data['launch_year'] = perfume_data['launch_year'].fillna(0)

def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return [] 


In [31]:
perfume_data['main_accords'] = perfume_data['main_accords'].fillna('[]')
perfume_data['notes'] = perfume_data['notes'].fillna('[]')

perfume_data['main_accords'] = perfume_data['main_accords'].apply(safe_literal_eval)
perfume_data['notes'] = perfume_data['notes'].apply(safe_literal_eval)

In [32]:
perfume_data['features'] = perfume_data['main_accords'].apply(lambda x: ' '.join(x)) + ' ' + perfume_data['notes'].apply(lambda x: ' '.join(x))

In [33]:
def recommend_perfume(user_liked_notes, n_recommendations=5):
    user_pref = ' '.join(user_liked_notes)

    count_vectorizer = CountVectorizer().fit_transform([user_pref] + perfume_data['features'].tolist())

    cosine_sim = cosine_similarity(count_vectorizer[0:1], count_vectorizer[1:]).flatten()

    similar_indices = cosine_sim.argsort()[-n_recommendations:][::-1]

    recommendations = perfume_data.iloc[similar_indices][['brand', 'perfume', 'main_accords', 'notes']]
    
    return recommendations, similar_indices


In [34]:
def test_recommendation_system(user_liked_notes):
    recommendations = recommend_perfume(user_liked_notes)
    
    print("Top recommended perfumes based on your preferences:")
    print(recommendations)

user_liked_notes = ['Honey', 'Woody', 'Vanilla']  
test_recommendation_system(user_liked_notes)

Top recommended perfumes based on your preferences:
(                 brand                perfume  \
6228         Leitmotiv  No 12 Figuier Moderne   
35684    4160 Tuesdays            Dirty Honey   
24960  Roberto Cavalli         Supreme Sandal   
5345     I Smell Great             Wild Honey   
37728         Fleurage           Wood Vanilla   

                                         main_accords  \
6228           [woody, sweet, fruity, vanilla, honey]   
35684  [honey, sweet, beeswax, amber, vanilla, woody]   
24960    [warm spicy, woody, honey, vanilla, powdery]   
5345         [vanilla, honey, sweet, powdery, floral]   
37728    [vanilla, woody, powdery, sweet, soft spicy]   

                                                   notes  
6228    [Fig, Passionfruit, Vanilla, Honey, Woody Notes]  
35684  [Honey, Beeswax, Vanilla, Woody Notes, Labdanu...  
24960   [Sandalwood, Honey, Saffron, Black Vanilla Husk]  
5345                           [Honey, Tahitian Vanilla]  
37728         